In [107]:
%%bash

# pip install --user pandas kfp kubeflow-metadata
# pip install Ipython --user
# pip install config --user
# pip install kubernetes --user
# pip install sklearn

In [108]:
# from ipython.display import display_html
# def restart_kernel():
#     display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
    
# restart_kernel()

In [109]:
import kfp.dsl as dsl
import kfp
import pandas as pd
import kfp.gcp as gcp
from kfp import components
import json
from string import Template
from config import Config
import os
import kubernetes
import yaml
import sklearn

In [110]:
with open("./train_image/train_template.yaml") as yml_in:
    b = yaml.safe_load(yml_in)
print(b)
# json.dumps(b)
train_temp = Template("""
    {
  "apiVersion": "batch/v1,",
  "kind": "Job",
  "metadata": {
    "name": "sklearn-job",
    "namespace": "$namespace"
  },
  "spec": {
    "template": {
      "spec": {
        "restartPolicy": "Never",
        "containers": [
          {
            "name": "sklearn-job",
            "image": "$image",
            "imagePullPolicy": "Always",
            "command": [
              "python /opt/model.py"
            ]
          }
        ]
      }
    }
  }
}

""")


{'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'name': '{{.Trial}}', 'namespace': '{{.Namespace}}'}, 'spec': {'template': {'spec': {'restartPolicy': 'Never', 'containers': [{'name': '{{.Trial}}', 'image': '$image', 'imagePullPolicy': 'Always', 'command': ['python /opt/model.py --tfmode=local {{- with .Hyperparameters}} {{- range .}} {{.Name}}={{.Value}} {{- end}} {{- end}}']}]}}}}


In [111]:
# trainjson = train_temp.substitute({'image': 'iamrock'})
# trainjson

In [112]:
json.dumps(trainjson)

'"\\n    {\\n  \\"apiVersion\\": \\"batch/v1,\\",\\n  \\"kind\\": \\"Job\\",\\n  \\"metadata\\": {\\n    \\"name\\": \\"{{.Trial}}\\",\\n    \\"namespace\\": \\"{{.Namespace}}\\"\\n  },\\n  \\"spec\\": {\\n    \\"template\\": {\\n      \\"spec\\": {\\n        \\"restartPolicy\\": \\"Never\\",\\n        \\"containers\\": [\\n          {\\n            \\"name\\": \\"{{.Trial}}\\",\\n            \\"image\\": \\"iamrock\\",\\n            \\"imagePullPolicy\\": \\"Always\\",\\n            \\"command\\": [\\n              \\"python /opt/model.py --tfmode=local {{- with .Hyperparameters}} {{- range .}} {{.Name}}={{.Value}} {{- end}} {{- end}}\\"\\n            ]\\n          }\\n        ]\\n      }\\n    }\\n  }\\n}\\n\\n"'

In [113]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/secret/gcp/user-gcp-sa.json


In [114]:
def convert_result(result) -> str:
    import json
    hyperparameters = json.loads(result)
    args = []
    print(type(hyperparameters))
    for key, value in hyperparameters.items():
        args.append("{0}={1}".format(key, value))
        
    return " ".join(args)





# x = '{"name":"John", "age":30, "city":"New York", "value": "none yet"}'
# convert_result(x)       

In [115]:
@dsl.pipeline(
    name="Police-Shootings-Justification-Prediction",
    description="A pipeline to train and serve police shootings justification in the United States"
)

def police_shootings_pipeline(
    name="police-shootings",
    katib_namespace="kubeflow-olawunmi1602",
    goal=0.9,
    max_trial_count=3,
    parallel_trial_count=3,
    training_steps=5,
    training_image="gcr.io/kubeflow-292422/police-shootings-train-sklearn:latest",
    training_namespace="kubeflow",
    serving_name="police-shootings-server",
    serving_namespace="kubeflow",
    serving_export_dir="gs://police-shootings/export",
    transformer_image="gcr.io/kubeflow-292422/police-shootings-processing:latest"
):
    
    objectiveConfig = {
        "type": "maximize",
        "goal": goal,
        "objectiveMetricName": "auc_score",
        "additionalMetricNames": ["loss", "accuracy"]
    }
    
    algorithmConfig = {
        "algorithmName": "random"
    }
    
    metricsCollectorSpec = {
        "collector": {
            "kind": "StdOut"
        }
    }
    
    parameters = [
        {"name": "--tf-learning-rate", "parameter-type": "double", "feasibleSpace": {"min": "0.001", "max": "0.005"}}
    ]
    
    train_template = train_temp
    train_json = train_template.substitute({'image': str(training_image), 'namespace': str(katib_namespace)})
    
    trialTemplate = {
        "goTemplate": {
            "rawTemplate": json.dumps(train_json)
        }
    }
    
    katib_experiment_launcher_op = components.load_component_from_url(
        'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml'
    )
    
    katib_op = katib_experiment_launcher_op(
        experiment_name=name,
        experiment_namespace=katib_namespace,
        parallel_trial_count=parallel_trial_count,
        max_trial_count=max_trial_count,
        objective=str(objectiveConfig),
        algorithm=str(algorithmConfig),
        trial_template=str(trialTemplate),
        parameters=str(parameters),
        metrics_collector=str(metricsCollectorSpec),
        delete_finished_experiment=False
    )
    
    convert_op = components.func_to_container_op(convert_result)
    op2 = convert_op(katib_op.output)

In [116]:
pipeline = kfp.Client().create_run_from_pipeline_func(police_shootings_pipeline, arguments={})

/home/jovyan/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Float" based on the value "0.9".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/home/jovyan/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "3".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/home/jovyan/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "5".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/home/jovyan/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:189: UserWarning: There are no registered serializers from type "bool" to type "Boo